In [46]:
import pandas as pd
import xarray as xr
import os
import matplotlib.pyplot as plt

BASE_DIR = "/Users/ct6g18/Python/UK_SAT_JAN"

data_dir = os.path.join(BASE_DIR,"Data")

filenames = os.listdir(data_dir)

filenames = sorted([os.path.join(data_dir,filename) for filename in filenames if not filename.startswith(".")])

filenames


['/Users/ct6g18/Python/UK_SAT_JAN/Data/tas_Amon_CSIRO-Mk3-6-0_rcp85_r1i1p1_200601-210012.nc',
 '/Users/ct6g18/Python/UK_SAT_JAN/Data/tas_Amon_CanESM2_rcp85_r1i1p1_200601-210012.nc',
 '/Users/ct6g18/Python/UK_SAT_JAN/Data/tas_Amon_GFDL-ESM2G_rcp85_r1i1p1_202101-202512.nc',
 '/Users/ct6g18/Python/UK_SAT_JAN/Data/tas_Amon_GFDL-ESM2G_rcp85_r1i1p1_202601-203012.nc',
 '/Users/ct6g18/Python/UK_SAT_JAN/Data/tas_Amon_GFDL-ESM2G_rcp85_r1i1p1_203101-203512.nc',
 '/Users/ct6g18/Python/UK_SAT_JAN/Data/tas_Amon_GFDL-ESM2G_rcp85_r1i1p1_203601-204012.nc',
 '/Users/ct6g18/Python/UK_SAT_JAN/Data/tas_Amon_GFDL-ESM2M_rcp85_r1i1p1_204101-204512.nc',
 '/Users/ct6g18/Python/UK_SAT_JAN/Data/tas_Amon_GFDL-ESM2M_rcp85_r1i1p1_204601-205012.nc',
 '/Users/ct6g18/Python/UK_SAT_JAN/Data/tas_Amon_GFDL-ESM2M_rcp85_r1i1p1_205101-205512.nc',
 '/Users/ct6g18/Python/UK_SAT_JAN/Data/tas_Amon_GFDL-ESM2M_rcp85_r1i1p1_205601-206012.nc',
 '/Users/ct6g18/Python/UK_SAT_JAN/Data/tas_Amon_HadGEM2-CC_rcp85_r1i1p1_200512-203011.nc',

In [47]:
airdata_IPSL   = xr.load_dataset(filenames[13])["tas"] - 273
airdata_GFDL   = xr.open_mfdataset(filenames[2:10])["tas"] - 273
airdata_HadGEM = xr.open_mfdataset(filenames[10:13])["tas"] - 273
airdata_CanESM = xr.load_dataset(filenames[1])["tas"] - 273
airdata_CSIRO  = xr.load_dataset(filenames[0])["tas"] - 273

In [48]:
airdatasets = {"IPSL"    : airdata_IPSL   
              ,"GFDL-ESM2G"    : airdata_GFDL   
              ,"HadGEM2"  : airdata_HadGEM 
              ,"CanESM2"  : airdata_CanESM 
              ,"CSIRO"   : airdata_CSIRO}

In [37]:
loc_coord_pairs = {"bristol" : {"lat": 51.458, "lon" :-2.60}
                  ,"london" : {"lat": 51.51, "lon" :-0.12}
                  ,"manchester" : {"lat": 53.48, "lon" :-2.24}
                  ,"edinburgh" : {"lat": 55.35, "lon" :-3.19}
                  ,"glasgow" : {"lat": 55.86, "lon" :-4.25}}

In [49]:
airdata_HadGEM

<xarray.DataArray 'tas' (time: 900, lat: 145, lon: 192)>
dask.array<sub, shape=(900, 145, 192), dtype=float32, chunksize=(300, 145, 192), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 2005-12-16 00:00:00 ... 2080-11-16 00:00:00
  * lat      (lat) float64 -90.0 -88.75 -87.5 -86.25 ... 86.25 87.5 88.75 90.0
  * lon      (lon) float64 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1
    height   float64 1.5

In [52]:
import numpy as np
ds_arr = np.zeros(shape=(5,5,120,3))

idx1_tags = {}
idx2_tags = {}

for idx1, (loc, coords) in enumerate(loc_coord_pairs.items()):
    for idx2, (model_name, model_data) in enumerate(airdatasets.items()):
        city_temp_mean_2030s = model_data.sel(**coords,method="nearest").sel(time = slice("2030-01","2039-12")).compute().values
        city_temp_mean_2040s = model_data.sel(**coords,method="nearest").sel(time = slice("2040-01","2049-12")).compute().values
        city_temp_mean_2050s = model_data.sel(**coords,method="nearest").sel(time = slice("2050-01","2059-12")).compute().values

        ds_arr[idx1,idx2,:,0] = city_temp_mean_2030s
        ds_arr[idx1,idx2,:,1] = city_temp_mean_2040s
        ds_arr[idx1,idx2,:,2] = city_temp_mean_2050s

        idx1_tags[idx1] = loc
        idx2_tags[idx2] = model_name

In [53]:
idx1_tags

{0: 'bristol', 1: 'london', 2: 'manchester', 3: 'edinburgh', 4: 'glasgow'}

In [54]:
idx2_tags

{0: 'IPSL', 1: 'GFDL-ESM2G', 2: 'HadGEM2', 3: 'CanESM2', 4: 'CSIRO'}

In [94]:
DJF_idx = [-1,0,1]
JJA_idx = [5,6,7]
for i in range(1,11):
    new_DJF_idx = [idx + 12 for idx in DJF_idx[-3:]]
    DJF_idx += new_DJF_idx

    new_JJA_idx = [idx + 12 for idx in JJA_idx[-3:]]
    JJA_idx += new_JJA_idx

DJF_idx = [idx for idx in DJF_idx if idx >= 0 and idx < 120]
JJA_idx = [idx for idx in JJA_idx if idx >= 0 and idx < 120]

ds_arr_DJF = ds_arr[:,:,DJF_idx,:]
ds_arr_JJA = ds_arr[:,:,JJA_idx,:]


np.shape(ds_arr_DJF)


DJF_means = np.mean(ds_arr_DJF,axis=2)
DJF_mins  = np.mean(ds_arr_DJF,axis=2)
DJF_maxs  = np.mean(ds_arr_DJF,axis=2)


JJA_means = np.mean(ds_arr_JJA,axis=2)
JJA_mins  = np.mean(ds_arr_JJA,axis=2)
JJA_maxs  = np.mean(ds_arr_JJA,axis=2)

np.shape(JJA_means)

mm_DJF_mean = np.mean(DJF_means,axis=1)
mm_DJF_max  = np.max(DJF_maxs,axis=1)
mm_DJF_min  = np.min(DJF_mins,axis=1)


winter_mean_df = pd.DataFrame(mm_DJF_mean,index=list(idx1_tags.values()),columns=["2030's DJF Mean","2040's DJF Mean","2050's DJF Mean"])
winter_min_df  = pd.DataFrame(mm_DJF_min, index=list(idx1_tags.values()),columns=["2030's DJF Min","2040's DJF Min","2050's DJF Min"])
winter_max_df  = pd.DataFrame(mm_DJF_max, index=list(idx1_tags.values()),columns=["2030's DJF Max","2040's DJF Max","2050's DJF Max"])


pd.concat([winter_max_df,winter_mean_df,winter_min_df],axis=1)

,2030's DJF Max,2040's DJF Max,2050's DJF Max,2030's DJF Mean,2040's DJF Mean,2050's DJF Mean,2030's DJF Min,2040's DJF Min,2050's DJF Min
bristol,7.026522,7.368494,7.643379,4.537651,4.999977,5.238904,-0.338069,1.041227,1.336692
london,7.026522,7.368494,7.643379,4.537651,4.999977,5.238904,-0.338069,1.041227,1.336692
manchester,8.500171,8.559320,9.093893,4.872986,5.469021,5.657679,0.555270,1.588270,1.921521
edinburgh,8.500171,8.559320,9.093893,5.070817,5.696183,5.873255,0.896849,1.883939,2.253428
glasgow,8.191591,8.327450,8.810556,5.093244,5.730715,5.874589,0.896849,1.883939,2.253428


In [95]:
JJA_means = np.mean(ds_arr_JJA,axis=2)
JJA_mins  = np.mean(ds_arr_JJA,axis=2)
JJA_maxs  = np.mean(ds_arr_JJA,axis=2)


JJA_means = np.mean(ds_arr_JJA,axis=2)
JJA_mins  = np.mean(ds_arr_JJA,axis=2)
JJA_maxs  = np.mean(ds_arr_JJA,axis=2)

np.shape(JJA_means)

mm_JJA_mean = np.mean(JJA_means,axis=1)
mm_JJA_max  = np.max(JJA_maxs,axis=1)
mm_JJA_min  = np.min(JJA_mins,axis=1)


winter_mean_df = pd.DataFrame(mm_JJA_mean,index=list(idx1_tags.values()),columns=["2030's JJA Mean","2040's JJA Mean","2050's JJA Mean"])
winter_min_df  = pd.DataFrame(mm_JJA_min, index=list(idx1_tags.values()),columns=["2030's JJA Min","2040's JJA Min","2050's JJA Min"])
winter_max_df  = pd.DataFrame(mm_JJA_max, index=list(idx1_tags.values()),columns=["2030's JJA Max","2040's JJA Max","2050's JJA Max"])


pd.concat([winter_max_df,winter_mean_df,winter_min_df],axis=1)

,2030's JJA Max,2040's JJA Max,2050's JJA Max,2030's JJA Mean,2040's JJA Mean,2050's JJA Mean,2030's JJA Min,2040's JJA Min,2050's JJA Min
bristol,20.869185,22.172801,22.431889,18.724314,19.584537,20.098738,15.682145,17.250506,17.232088
london,20.869185,22.172801,22.431889,18.724314,19.584537,20.098738,15.682145,17.250506,17.232088
manchester,17.973073,18.914030,19.130721,16.584842,17.251138,17.627795,14.075637,15.231160,15.209266
edinburgh,16.852702,16.984429,17.623011,14.978306,15.317513,15.552393,12.759741,12.701610,12.653162
glasgow,16.655698,16.726093,17.235184,14.771306,15.091534,15.302100,12.759741,12.701610,12.653162
